# Check Cointegration for 10 clusters - found by K-Medoids

In [1]:
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
import pandas as pd
import numpy as np
import itertools
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.pylab as pit
from matplotlib.pylab import rcParams
%matplotlib inline
from matplotlib import style
style.use('ggplot')
rcParams['figure.figsize'] = 20,10

C:\Users\archana.parihar\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [4]:
df = pd.DataFrame()
df = pd.read_csv('C:/Users/archana.parihar/Downloads/ClustersFromR_10_K-Medoids.csv')
dfCluster1 = df.loc[df['Cluster'] == 1]
dfCluster2 = df.loc[df['Cluster'] == 2]
dfCluster3 = df.loc[df['Cluster'] == 3]
dfCluster4 = df.loc[df['Cluster'] == 4]
dfCluster5 = df.loc[df['Cluster'] == 5]
dfCluster6 = df.loc[df['Cluster'] == 6]
dfCluster7 = df.loc[df['Cluster'] == 7]
dfCluster8 = df.loc[df['Cluster'] == 8]
dfCluster9 = df.loc[df['Cluster'] == 9]
dfCluster10 = df.loc[df['Cluster'] == 10]


In [5]:
badCompanies = ['BMY', 'DHR', 'ES', 'ICE', 'ORCL', 'O', 'IQV', 'COTY', 'FOXA', 'FOX', 'NWSA', 'NWS', 'ALLE', 'GOOG', 'NAVI', 'INFO', 'SYF', 'CFG', 'QRVO', 'WRK', 'KHC', 'PYPL', 'HPE', 'HPQ', 'CSRA', 'WLTW', 'UA', 'FTV', 'EVHC', 'HLT', 'DXC', 'BHGE', 'BHF', 'DWDP', 'APTV', 'CBG', 'CHK', 'HCN', 'LUK', 'PCLN', 'PDCO', 'SIG', 'SNI', 'WYN', 'CSRA', 'ABMD', 'BKNG', 'CBRE', 'IPGP', 'JEF', 'MSCI', 'NKTR', 'SIVB', 'TTWO', 'WELL']

In [6]:
df1 = pd.DataFrame()
df1 = pd.read_csv('C:/Users/archana.parihar/Downloads/allStocks13-14-15.csv', parse_dates=True,index_col=0)


In [7]:
def test_CointegrationOfCombinedSeries(timeSeries):
    coint = False    
    #plot rolling statistics to visualize whether it is stationary or not
    ma = timeSeries.rolling(window=100).mean()
    mstd = timeSeries.rolling(window=100).std()
    
   # timeSeries.plot()
   # ma.plot(color='blue')
   # mstd.plot(color='black')
    
    #perform dickey fuller to check quantitatively
    dftest = adfuller(timeSeries,autolag= 'AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#lags used','No of observations used'])
    for key,value in dftest[4].items():
        dfoutput['Critical value (%s)'%key]= value

    #print(dfoutput) 
    #if dfoutput['Critical value (10%)']<-2.57  :
       # print('The pair is most likely Cointegrated')
        #coint = True
    #if dfoutput['Critical value (10%)']>-2.57  :
        #print('The pair is most likely NOT Cointegrated')
    if dfoutput['p-value']<=0.01:
        coint= True
    return coint

def ImproveStationarity_and_test_Cointegration(dfX,dfY):
    #Convert closePrice of Time series X to log-Moving average time series for better value of regression coefficient
    dfX_logScale = np.log(dfX['close'])
    Xma = dfX_logScale.rolling(window=100).mean()
    logScaleMinusMovingAverage1 = dfX_logScale - Xma
    logScaleMinusMovingAverage1.dropna(inplace=True)

    #Convert closePrice of Time series Y to log-Moving average time series for better value of regression coefficient
    dfY_logScale = np.log(dfY['close'])
    Yma = dfY_logScale.rolling(window=100).mean()
    logScaleMinusMovingAverage2 = dfY_logScale - Yma
    logScaleMinusMovingAverage2.dropna(inplace=True)

    #Using linear regression to calculate the value of b for linearly combines Time series
    logScaleMinusMovingAverage1 = sm.add_constant(logScaleMinusMovingAverage1)
    results = sm.OLS(logScaleMinusMovingAverage2,logScaleMinusMovingAverage1['close']).fit()
    b = results.params['close']
    z = logScaleMinusMovingAverage1['close'] - (b * logScaleMinusMovingAverage2)
    coint = test_CointegrationOfCombinedSeries(z)
    return coint

In [8]:
def coIntTestForCluster(dfCluster,ClusterNumber):
    print('Cluster number: '+ str(ClusterNumber))
    ListOfCompInOneCluster = dfCluster['Name'].tolist()
    print('Old length of cluster : '+ str(len(ListOfCompInOneCluster)))
    for i in range(0,len(badCompanies)):
        if badCompanies[i] in ListOfCompInOneCluster :
            #print("Bad Company found in Cluster: " , badCompanies[i])
            ListOfCompInOneCluster.remove(badCompanies[i])
    print('New length of cluster : '+ str(len(ListOfCompInOneCluster)))

    ListOfPairs = list(itertools.combinations(ListOfCompInOneCluster, 2))
    #print(ListOfPairs[:2])
    #print(len(ListOfPairs))
    #print(ListOfPairs[0][0],ListOfPairs[0][1])

    pair_result_dict = {}
    for x in range(0,len(ListOfPairs)):
        pair_result_dict[ListOfPairs[x]] = []

    for i in range(0,len(ListOfPairs)):
        dfX = df1.loc[df1['Name']==ListOfPairs[i][0]]
        dfY = df1.loc[df1['Name']==ListOfPairs[i][1]]
        Result = ImproveStationarity_and_test_Cointegration(dfX,dfY)
        pair_result_dict[ListOfPairs[i]] = Result

    ListOfValuesCluster_2=list(pair_result_dict.values())
    print('Total pairs:  ' + str(len(ListOfValuesCluster_2)))
    print('Number of True: ' + str(ListOfValuesCluster_2.count(True)))
    print('Number of False: ' + str(ListOfValuesCluster_2.count(False)))
    print('---------------------------------------------------------------')
    return pair_result_dict

listOfClusters = [dfCluster1,dfCluster2,dfCluster3,dfCluster4,dfCluster5,dfCluster6,dfCluster7,dfCluster8,dfCluster9,dfCluster10]
listOfResultDict = []

for c in range(0,len(listOfClusters)):
    res = coIntTestForCluster(listOfClusters[c],c)
    listOfResultDict.append(res)

Cluster number: 0
Old length of cluster : 62
New length of cluster : 56
Total pairs:  1540
Number of True: 120
Number of False: 1420
---------------------------------------------------------------
Cluster number: 1
Old length of cluster : 67
New length of cluster : 64
Total pairs:  2016
Number of True: 239
Number of False: 1777
---------------------------------------------------------------
Cluster number: 2
Old length of cluster : 82
New length of cluster : 73
Total pairs:  2628
Number of True: 303
Number of False: 2325
---------------------------------------------------------------
Cluster number: 3
Old length of cluster : 72
New length of cluster : 63
Total pairs:  1953
Number of True: 186
Number of False: 1767
---------------------------------------------------------------
Cluster number: 4
Old length of cluster : 34
New length of cluster : 32
Total pairs:  496
Number of True: 30
Number of False: 466
---------------------------------------------------------------
Cluster number: 5


In [9]:
ListOfCointPairs = []
for key, value in listOfResultDict[0].items():
    if value == True:
        ListOfCointPairs.append(key)
        
print(len(ListOfCointPairs))

dfCoInt = pd.DataFrame({'Pairs':ListOfCointPairs})
dfCoInt.to_csv('C:/Users/archana.parihar/Downloads/ResultsSnP/ListOfPairs_10_Medoid_Cluster_1_Pvalue_01.csv',index=False, sep=',',encoding='utf-8')

120


In [10]:
ListOfCointPairs = []

for key, value in listOfResultDict[1].items():
    if value == True:
        ListOfCointPairs.append(key)
        
print(len(ListOfCointPairs))

dfCoInt = pd.DataFrame({'Pairs':ListOfCointPairs})
dfCoInt.to_csv('C:/Users/archana.parihar/Downloads/ResultsSnP/ListOfPairs_10_Medoid_Cluster_2_Pvalue_01.csv',index=False, sep=',',encoding='utf-8')

239


In [11]:
ListOfCointPairs = []

for key, value in listOfResultDict[2].items():
    if value == True:
        ListOfCointPairs.append(key)
        
print(len(ListOfCointPairs))

dfCoInt = pd.DataFrame({'Pairs':ListOfCointPairs})
dfCoInt.to_csv('C:/Users/archana.parihar/Downloads/ResultsSnP/ListOfPairs_10_Medoid_Cluster_3_Pvalue_01.csv',index=False, sep=',',encoding='utf-8')

303


In [12]:
ListOfCointPairs = []

for key, value in listOfResultDict[3].items():
    if value == True:
        ListOfCointPairs.append(key)
        
print(len(ListOfCointPairs))

dfCoInt = pd.DataFrame({'Pairs':ListOfCointPairs})
dfCoInt.to_csv('C:/Users/archana.parihar/Downloads/ResultsSnP/ListOfPairs_10_Medoid_Cluster_4_Pvalue_01.csv',index=False, sep=',',encoding='utf-8')

186


In [13]:
ListOfCointPairs = []

for key, value in listOfResultDict[4].items():
    if value == True:
        ListOfCointPairs.append(key)
        
print(len(ListOfCointPairs))

dfCoInt = pd.DataFrame({'Pairs':ListOfCointPairs})
dfCoInt.to_csv('C:/Users/archana.parihar/Downloads/ResultsSnP/ListOfPairs_10_Medoid_Cluster_5_Pvalue_01.csv',index=False, sep=',',encoding='utf-8')

30


In [14]:
ListOfCointPairs = []

for key, value in listOfResultDict[5].items():
    if value == True:
        ListOfCointPairs.append(key)
        
print(len(ListOfCointPairs))

dfCoInt = pd.DataFrame({'Pairs':ListOfCointPairs})
dfCoInt.to_csv('C:/Users/archana.parihar/Downloads/ResultsSnP/ListOfPairs_10_Medoid_Cluster_6_Pvalue_01.csv',index=False, sep=',',encoding='utf-8')

33


In [15]:
ListOfCointPairs = []

for key, value in listOfResultDict[6].items():
    if value == True:
        ListOfCointPairs.append(key)
        
print(len(ListOfCointPairs))

dfCoInt = pd.DataFrame({'Pairs':ListOfCointPairs})
dfCoInt.to_csv('C:/Users/archana.parihar/Downloads/ResultsSnP/ListOfPairs_10_Medoid_Cluster_7_Pvalue_01.csv',index=False, sep=',',encoding='utf-8')

286


In [16]:
ListOfCointPairs = []

for key, value in listOfResultDict[7].items():
    if value == True:
        ListOfCointPairs.append(key)
        
print(len(ListOfCointPairs))

dfCoInt = pd.DataFrame({'Pairs':ListOfCointPairs})
dfCoInt.to_csv('C:/Users/archana.parihar/Downloads/ResultsSnP/ListOfPairs_10_Medoid_Cluster_8_Pvalue_01.csv',index=False, sep=',',encoding='utf-8')

68


In [17]:
ListOfCointPairs = []

for key, value in listOfResultDict[8].items():
    if value == True:
        ListOfCointPairs.append(key)
        
print(len(ListOfCointPairs))

dfCoInt = pd.DataFrame({'Pairs':ListOfCointPairs})
dfCoInt.to_csv('C:/Users/archana.parihar/Downloads/ResultsSnP/ListOfPairs_10_Medoid_Cluster_9_Pvalue_01.csv',index=False, sep=',',encoding='utf-8')

19


In [18]:
ListOfCointPairs = []

for key, value in listOfResultDict[9].items():
    if value == True:
        ListOfCointPairs.append(key)
        
print(len(ListOfCointPairs))

dfCoInt = pd.DataFrame({'Pairs':ListOfCointPairs})
dfCoInt.to_csv('C:/Users/archana.parihar/Downloads/ResultsSnP/ListOfPairs_10_Medoid_Cluster_10_Pvalue_01.csv',index=False, sep=',',encoding='utf-8')

45
